In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:85% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="6" color="red">ch15. DB연동</font></b>
# 1. SQLite DB 연결
- SQLite DB는 별도의 DBMS 없이 SQL문을 이용하여 DB에 액세스할 수 있도록 만든 간단한 디스크
- C라이브러리
- SQLite는 프로토타입 생성 시 사용
- 프로젝트 : 분석 -> 설계 -> 구현 -> 테스트 -> 고객에게 배포 -> 유지/보수 
             프로토타입(SQLite)  완제품(Oracle, MySQL, Maria, ...)
- [DB browser for SQLite](https:/sqlitebrowser.org/dl/)
## 1.1.SQLite browser 설치 및 sqlite 패키지 load

In [2]:
import sqlite3
sqlite3.sqlite_version # 버전

'3.40.1'

In [3]:
import pandas as pd
pd.__version__

'1.5.3'

## 1.2.DB 연결

- (1) DB 연결 객체 생성
- (2) SQL 전송 객체 생성(cursor)
- (3) SQL 전송 & 결과 받기(cursor.execute() 이용)
- (4) cursor 해제 & DB 연결 객체 해제

In [4]:
# (1) DB 연결 : sqlite로 연결 시 DB 파일이 없으면 빈 DB 파일을 생성 및 연결하고 있으면 그냥 연결
conn  = sqlite3.connect('data/ch15_example.db')
conn

In [5]:
# (2) SQL 전송 객체 생성(cursor 객체)
cursor = conn.cursor()
cursor

In [6]:
# (3) SQL 전송 & 결과 받기(cursor.execute() 이용)
cursor.execute('''
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
''')

In [7]:
cursor.execute('DROP TABLE MEMBER')

In [8]:
cursor.execute('''
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
''')

In [14]:
cursor.execute('INSERT INTO MEMBER VALUES (\'홍길동\', 20, \'h@h.com\')')
print('수행 결과 행수 :', cursor.rowcount)

수행 결과 행수 : 1


In [15]:
sql = "INSERT INTO MEMBER VALUES ('김길동', 30, 'K@K.COM')"
cursor.execute(sql)
print('수행결과의 행수 :', cursor.rowcount)
cursor.execute("INSERT INTO MEMBER VALUES ('이길동', 25, 'l@l.com')")
print('수행결과의 행수 :', cursor.rowcount)

수행결과의 행수 : 1
수행결과의 행수 : 1


In [16]:
conn.commit() # < -- > conn.rollback()

In [17]:
# select문 전송. 전송 결과 : cursor가 가리킴
cursor.execute("SELECT * FROM MEMBER")

In [18]:
# insert, update, delete문 실행 결과 조회: cursor.rowcount
# select문 실행 결과 조회 : 
        ## fetchall() : 결과를 모두 받을 때 (튜플 list)
        ## fetchone() : 결과를 한 행씩 받을 때 (튜플)
        ## fetchmany(n) : 결과를 n행 받을 때 (튜플 list)
print(cursor.fetchall())

[('홍길동', 20, 'h@h.com'), ('김길동', 30, 'K@K.COM'), ('이길동', 25, 'l@l.com')]


In [19]:
print(cursor.fetchall()) # 한 번 소모된 cursor 객체는 다시 fetch될 수 없으므로 변수에 담는 것이 안전

[]


In [20]:
# 한꺼번에 읽어오기
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
members = cursor.fetchall()
members

[('홍길동', 20, 'h@h.com'), ('이길동', 25, 'l@l.com'), ('김길동', 30, 'K@K.COM')]

In [21]:
for member in members:
    print(member)

('홍길동', 20, 'h@h.com')
('이길동', 25, 'l@l.com')
('김길동', 30, 'K@K.COM')


In [23]:
# 하나씩 읽기
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
members = []
while True:
    member = cursor.fetchone()
    if member is None:
        print('데이터 끝')
        break
    members.append(member)

데이터 끝


In [24]:
members

[('홍길동', 20, 'h@h.com'), ('이길동', 25, 'l@l.com'), ('김길동', 30, 'K@K.COM')]

In [25]:
# 최상위 n행 읽기
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
for member in cursor.fetchmany(2):
    print(member)

('홍길동', 20, 'h@h.com')
('이길동', 25, 'l@l.com')


In [26]:
cursor.description

(('NAME', None, None, None, None, None, None),
 ('AGE', None, None, None, None, None, None),
 ('EMAIL', None, None, None, None, None, None))

In [44]:
class Member:
    'Member 테이블의 내용을 받을 객체 타입'
    def __init__(self, name, age, email):
        self.name = name 
        self.age = age
        self.email = email
    def __str__(self):
        return '{}\t{}\t{}'.format(self.name, self.age, self.email)
def to_member(*row): # 튜플 매개변수를 받아 Member형 객체 return
    return Member(row[0], row[1], row[2])

In [36]:
cursor.execute("SELECT * FROM MEMBER")
member_list = [] # sql문 수행한 결과를 담을 객체 list
members = cursor.fetchall() # 튜플 list
for member in members :
    member_list.append(to_member(*member))

In [37]:
for member in member_list:
    print(member)

홍길동	20	h@h.com
김길동	30	K@K.COM
이길동	25	l@l.com


In [39]:
# (4) 연결 해제
cursor.close()
conn.close()

## 1.3.SQL 구문에 파라미터 사용하기
- qmark(DB에 따라 불가한 경우 있음)
- named(추천)

In [40]:
conn = sqlite3.connect('data/ch15_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('홍길동', '김길동')")
cursor.fetchall()

[('홍길동', 20, 'h@h.com'), ('김길동', 30, 'K@K.COM')]

In [42]:
# 파라미터 사용하기 : qmark 방법
name1 = input('검색할 이름1?')
name2 = input('검색할 이름2?')
# cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('"+name1+"', '"+name2+"')")
# cursor.execute(f"SELECT * FROM MEMBER WHERE NAME IN ('{name1}', '{name2}')")
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?, ?)", (name1, name2))
cursor.fetchall()

검색할 이름1?홍길동
검색할 이름2?이길동


[('홍길동', 20, 'h@h.com'), ('이길동', 25, 'l@l.com')]

In [43]:
# 파라미터 사용하기 : named 방법
name1 = input('검색할 이름1?')
name2 = input('검색할 이름2?')
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (:name1, :name2)", {'name1':name1,
                                                                       'name2':name2})
cursor.fetchall()

검색할 이름1?홍길동
검색할 이름2?김길동


[('홍길동', 20, 'h@h.com'), ('김길동', 30, 'K@K.COM')]

In [ ]:
class Member:
    'Member 테이블의 내용을 받을 객체 타입'
    def __init__(self, name, age, email):
        self.name = name 
        self.age = age
        self.email = email
    def __str__(self):
        return '{}\t{}\t{}'.format(self.name, self.age, self.email)
def to_member(*row): # 튜플 매개변수를 받아 Member형 객체 return
    return Member(row[0], row[1], row[2])

In [46]:
# Member 테이블에 입력 (사용자로부터 이름, 나이, 메일을 받아 insert)
try:
    name = input('입력할 이름 :')
    age = int(input('입력할 나이 :'))
except:
    print('유효하지 않은 나이를 입력한 경우, 18세로 초기화')
    age = 18
finally:
    email = input('입력할 메일은 :')
inputdata = {'name':name, 'age':age, 'email':email} # named 방식에서 사용
# inputdata = (name, age, email) # qmark 방식에서 사용
# sql = "INSERT INTO MEMBER VALUES (?, ?, ?)", inputdata)
sql = "INSERT INTO MEMBER VALUES (:name, :age, :email)"
cursor.execute(sql, inputdata)
conn.commit()
if cursor.rowcount:
    print('저장 완료')

입력할 이름 :마길동
입력할 나이 :19
입력할 메일은 :m@ma.com
저장 완료


In [48]:
# Member 테이블에 입력 (사용자로부터 이름, 나이, 메일을 받아 insert)
try:
    name = input('입력할 이름 :')
    age = int(input('입력할 나이 :'))
except:
    print('유효하지 않은 나이를 입력한 경우, 18세로 초기화')
    age = 18
finally:
    email = input('입력할 메일은 :')
newMember = Member(name, age, email)
sql = "INSERT INTO MEMBER VALUES (:name, :age, :email)"
cursor.execute(sql, newMember.__dict__)
conn.commit()
if cursor.rowcount==1:
    print('입력 성공')
# print(newMember)
# print(newMember.__dict__)

입력할 이름 :신길동
입력할 나이 :22
입력할 메일은 :s@sin.com
입력 성공


In [50]:
while True:
    try:
        name = input('입력할 이름(종료는 0) :')
        if name=='0':
            break;
        age = int(input('입력할 나이 :'))
    except:
        print('유효하지 않은 나이를 입력한 경우, 18세로 초기화')
        age = 18
    email = input('입력할 메일은 :')
    newMember = Member(name, age, email)
    sql = "INSERT INTO MEMBER VALUES (:name, :age, :email)"
    cursor.execute(sql, newMember.__dict__)
    if cursor.rowcount==1:
        print('입력 성공')
conn.commit()

입력할 이름(종료는 0) :김길호
입력할 나이 :39
입력할 메일은 :kh@kim.com
입력 성공
입력할 이름(종료는 0) :장수호
입력할 나이 :36
입력할 메일은 :j@jang.com
입력 성공
입력할 이름(종료는 0) :0


In [51]:
cursor.close()
conn.close()

# 2. oracle DB 연동
- pip install cx_oracle (~oracle 11g)
- pip install oracledb(oracle 12c~)

In [52]:
import cx_Oracle
cx_Oracle.__version__

'8.3.0'

In [53]:
# conn 객체 얻어오는 방법1
conn = cx_Oracle.connect('scott', 'tiger', 'localhost:1521/xe')
conn

<cx_Oracle.Connection to scott@localhost:1521/xe>

In [54]:
cursor = conn.cursor()

In [55]:
sql = 'SELECT EMPNO "NO", ENAME, JOB, MGR, HIREDATE, SAL, COMM, DEPTNO FROM EMP'
cursor.execute(sql)
cursor.fetchall()

[(7369,
  'SMITH',
  'CLERK',
  7902,
  datetime.datetime(1980, 12, 17, 0, 0),
  800.0,
  None,
  20),
 (7499,
  'ALLEN',
  'SALESMAN',
  7698,
  datetime.datetime(1981, 2, 20, 0, 0),
  1600.0,
  300.0,
  30),
 (7521,
  'WARD',
  'SALESMAN',
  7698,
  datetime.datetime(1981, 2, 22, 0, 0),
  1250.0,
  500.0,
  30),
 (7566,
  'JONES',
  'MANAGER',
  7839,
  datetime.datetime(1981, 4, 2, 0, 0),
  2975.0,
  None,
  20),
 (7654,
  'MARTIN',
  'SALESMAN',
  7698,
  datetime.datetime(1981, 9, 28, 0, 0),
  1250.0,
  1400.0,
  30),
 (7698,
  'BLAKE',
  'MANAGER',
  7839,
  datetime.datetime(1981, 5, 1, 0, 0),
  2850.0,
  None,
  30),
 (7782,
  'CLARK',
  'MANAGER',
  7839,
  datetime.datetime(1981, 6, 9, 0, 0),
  2450.0,
  None,
  10),
 (7788,
  'SCOTT',
  'ANALYST',
  7566,
  datetime.datetime(1982, 12, 9, 0, 0),
  3000.0,
  None,
  20),
 (7839,
  'KING',
  'PRESIDENT',
  None,
  datetime.datetime(1981, 11, 17, 0, 0),
  5000.0,
  None,
  10),
 (7844,
  'TURNER',
  'SALESMAN',
  7698,
  datetim

In [2]:
# conn 얻어 오는 방법2
import oracledb
oracledb.init_oracle_client()
# conn = oracledb.connect('scott/tiger@localhost:1521/xe')
conn = oracledb.connect(
    user = 'scott',
    password = 'tiger',
    host = 'localhost',
    port = 1521,
    sid = 'xe'
)
cursor = conn.cursor()
sql = 'SELECT EMPNO "NO", ENAME, JOB, MGR, HIREDATE, SAL, COMM, DEPTNO FROM EMP'
emp = cursor.execute(sql)
cursor.fetchall()
print(emp)

<oracledb.Cursor on <oracledb.Connection to scott@(DESCRIPTION=(ADDRESS=(PROTOCOL=tcp)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>>
